<a href="https://colab.research.google.com/github/kyrafka/sesion03-aes-rsaV2/blob/main/Sesion_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🚀 INSTALAR DEPENDENCIAS
!pip install pycryptodome

# 📌 IMPORTAR LIBRERÍAS
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
from Crypto.PublicKey import RSA
import base64

# 🔹 1️⃣ GENERAR CLAVE AES Y CIFRAR TARJETA
tarjeta_credito = "123345567"  # Ejemplo de número de tarjeta

# Generar clave AES de 256 bits (32 bytes)
clave_aes = get_random_bytes(32)

# Crear cifrador AES en modo CBC
iv = get_random_bytes(16)  # Vector de inicialización
cipher = AES.new(clave_aes, AES.MODE_CBC, iv)

# Cifrar los datos con padding
datos_cifrados = cipher.encrypt(pad(tarjeta_credito.encode(), AES.block_size))

# Convertir a Base64 para almacenar seguro
datos_cifrados_b64 = base64.b64encode(iv + datos_cifrados).decode()
print("🔐 Tarjeta cifrada (Base64):", datos_cifrados_b64)

# 🔹 2️⃣ GENERAR CLAVES RSA Y CIFRAR LA CLAVE AES
# Generar clave RSA de 2048 bits
clave_rsa = RSA.generate(2048)
clave_publica = clave_rsa.publickey()

# Cifrar la clave AES con RSA
cipher_rsa = PKCS1_OAEP.new(clave_publica)
clave_aes_cifrada = cipher_rsa.encrypt(clave_aes)

# Convertir la clave AES cifrada a Base64
clave_aes_cifrada_b64 = base64.b64encode(clave_aes_cifrada).decode()
print("🔒 Clave AES cifrada con RSA (Base64):", clave_aes_cifrada_b64)

# 🔹 3️⃣ DESCIFRAR LA CLAVE AES CON RSA
cipher_rsa_descifrado = PKCS1_OAEP.new(clave_rsa)
clave_aes_descifrada = cipher_rsa_descifrado.decrypt(base64.b64decode(clave_aes_cifrada_b64))

print("🔓 Clave AES descifrada:", clave_aes_descifrada.hex())

# 🔹 4️⃣ DESCIFRAR LA TARJETA DE CRÉDITO CON AES
# Extraer IV y datos cifrados
datos_cifrados_decoded = base64.b64decode(datos_cifrados_b64)
iv_recuperado = datos_cifrados_decoded[:16]
datos_cifrados_real = datos_cifrados_decoded[16:]

# Crear descifrador AES con la clave recuperada
cipher_dec = AES.new(clave_aes_descifrada, AES.MODE_CBC, iv_recuperado)

# Descifrar y quitar padding
tarjeta_descifrada = unpad(cipher_dec.decrypt(datos_cifrados_real), AES.block_size).decode()

print("💳 Tarjeta de crédito descifrada:", tarjeta_descifrada)


🔐 Tarjeta cifrada (Base64): dCXDNKCZuQKTbsXHJSD/DNxiQIllwMMfgwCigap8x+0=
🔒 Clave AES cifrada con RSA (Base64): n+HlQgcfOZfJx7awyCtpOL7og/8w+uOvbwsnQ3HxI1oCYTDT5MJBNIds44voJxF9vjULHKAO8szL3LVOexXCthKEVItQjv+U+Kglae2zoCxzq5G9NVSPwOnGcaWf9UW3gf/Xc0hOcEttr8sdhqHt1AEwo5j5JgsbeCy8+SlSwvF0F3s1C0xLNo5ZZJqt2zTQgyAlsFseZAavlTts2+Jnf6YrpyFRfyuMOcf3gxD62KbOWaGiuse6AV9LUes6OKEzNDHmQygZ1UglO5VGTr8IaJzhTEf8EYhD/5SL+n22TQYiPuJjDs8SHWssA0BPV57FcrNMEcx5F8S8hdpoXXRnwQ==
🔓 Clave AES descifrada: a4a8a61d0741096ded22af2d33d3aa8c94835fbb6bdb0e11e0db6697a513c131
💳 Tarjeta de crédito descifrada: 123345567
